<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Extra/01_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

El *web scraping* es un proceso mediante el cual podemos extraer datos de distintos sitios web. Python tiene muchas herramientas para realizarlo de manera rápida, algunas de las cuales veremos en esta práctica.

# Páginas estáticas

Una página estática es aquella que muestra el mismo contenido a todos los usuarios, y utilizan principalmente HTML. Estas páginas son muy fáciles de scrapear, por lo cual empezaremos con ellas.

La página con la que estaremos trabajando es https://www.scrapethissite.com/pages/forms/. Esta contiene datos de distintos equipos de Hockey en una tabla, los cuales queremos transformar a una DataFrame de Pandas.

Empezamos descargando la página utilizando `requests.get`:

In [125]:
import requests

page = requests.get("https://www.scrapethissite.com/pages/forms/")

Si todo está bien, al imprimir el objeto deberíamos de ver un código de respuesta `<Response [200]>`

In [126]:
print(page)

<Response [200]>


El HTML completo de la página está en el atributo `content` del objeto:

In [ ]:
print(page.content)

Si bien en principio podríamos procesar esto con expresiones regulares u otra técnica, en la práctica esto es muy difícil. Es aquí donde entra `BeautifulSoup`:

In [128]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content)

In [130]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>
<link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
<link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
<meta content="noindex" name="robot

Esta es una herramienta muy poderosa que automáticamente analiza el contenido del HTML. Podemos verlo accesando el atributo `contents`:

In [ ]:
soup.contents

`bs4` adicionalmente proporciona muchos métodos para descargar partes específicas de la página. Una de estas es `find`, que regresa el primer resultado que se ajusta a un patrón de búsqueda. Estos patrones pueden basarse en la clase, id o etiqueta del objeto, y mucho más.

Por ejemplo, si queremos extraer el título, podemos utilizar el inspector de nuestro navegador para determinar qué tipo de objeto es:

![clase](./scrape_01.png)

Podemos ver que tiene etiqueta `h1`, así que la pasamos como argumento a `find`:

In [1]:
head = soup.find_all('h1')
head

NameError: name 'soup' is not defined

Esto nos regresa a su vez un objeto de `bs4`. Si queremos accesar el texto, simplemente consultamos el atributo `text`:

In [134]:
head.text.strip()

'Hockey Teams: Forms, Searching and Pagination\n                            25 items'

Ahora, en HTML las tablas suelen estar organizadas por capas de la siguiente manera:

```
<table>
    <tr>
        <th>encabezado_1</th> <th>encabezado_2</th> ... <th>encabezado_1</th>
    </tr>
    <tr>
        <td>celda_1</td> <td>celda_2</td> ... <td>celda_n</td>
    </tr>
    <tr>
        <td> <td> ... <td>
    </tr>
    ...
    <tr>
        <td> <td> ... <dt>
    </tr>
</table>
```

La etiqueta `<tr>` indica una fila de la tabla, y la `<td>` cada una de las celdas correspondiente a esa fila y columna. Por lo tanto, lo primero que tenemos que hacer es buscar la etiqueta de `table`:

In [135]:
table = soup.find('table')
table

<table class="table">
<tr>
<th>
                            Team Name
                        </th>
<th>
                            Year
                        </th>
<th>
                            Wins
                        </th>
<th>
                            Losses
                        </th>
<th>
                            OT Losses
                        </th>
<th>
                            Win %
                        </th>
<th>
                            Goals For (GF)
                        </th>
<th>
                            Goals Against (GA)
                        </th>
<th>
                            + / -
                        </th>
</tr>
<tr class="team">
<td class="name">
                            Boston Bruins
                        </td>
<td class="year">
                            1990
                        </td>
<td class="wins">
                            44
                        </td>
<td class="losses">
                            2

Luego, obtenemos todas las filas utilizando `find_all`. Esto nos regresa una lista, donde cada entrada es un objeto de `bs4` con la correspondiente coincidencia:

In [141]:
rows = table.find_all('tr')
rows[5]

<tr class="team">
<td class="name">
                            Detroit Red Wings
                        </td>
<td class="year">
                            1990
                        </td>
<td class="wins">
                            34
                        </td>
<td class="losses">
                            38
                        </td>
<td class="ot-losses">
</td>
<td class="pct text-danger">
                            0.425
                        </td>
<td class="gf">
                            273
                        </td>
<td class="ga">
                            298
                        </td>
<td class="diff text-danger">
                            -25
                        </td>
</tr>

Finalmente, iteramos sobre cada fila y obtenemos todas las celdas. Por ejemplo, para la segunda fila:

In [148]:
test = rows[1].find_all('td')
test[0]

<td class="name">
                            Boston Bruins
                        </td>

El texto correspondiente a cada celda está en el atributo `text`:

In [150]:
test[0].text

'\n                            Boston Bruins\n                        '

Podemos limpiarlo para quitar todo el espacio extra y los saltos de línea:

In [154]:
test[0].text.strip()

'Boston Bruins'

In [155]:
out = []
for row in rows:
    cells = row.find_all('td')
    cells_text = [cell.text.strip() for cell in cells]
    out.append(cells_text)

Viendo nuestros resultados:

In [159]:
print(out[0])
print()
print(out[1])
print(out[2])
print(out[3])

[]

['Boston Bruins', '1990', '44', '24', '', '0.55', '299', '264', '35']
['Buffalo Sabres', '1990', '31', '30', '', '0.388', '292', '278', '14']
['Calgary Flames', '1990', '46', '26', '', '0.575', '344', '263', '81']


Podemos ver que la primera fila de la tabla (correspondiente al encabezado) no se guardó correctamente. Esto se debe a que las celdas en el encabezado están etiquetadas con `<th>`. Podemos obtenerlas a mano:

In [161]:
header = rows[0].find_all('th')
out[0] = [cell.text.strip() for cell in header]

Convirtiendo nuestra lista a una DataFrame:

In [162]:
df = pd.DataFrame(out[1:], columns=out[0])
df

,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,,0.55,299,264,35
1,Buffalo Sabres,1990,31,30,,0.388,292,278,14
2,Calgary Flames,1990,46,26,,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,,0.425,273,298,-25
5,Edmonton Oilers,1990,37,37,,0.463,272,272,0
6,Hartford Whalers,1990,31,38,,0.388,238,276,-38
7,Los Angeles Kings,1990,46,24,,0.575,340,254,86
8,Minnesota North Stars,1990,27,39,,0.338,256,266,-10
9,Montreal Canadiens,1990,39,30,,0.487,273,249,24


# Páginas dinámicas

Las páginas dinámicas son más difíciles de scrapear, ya que su contenido no está incrustado en el HTML, sino que se genera dependiendo del usuario utilizando JavaScript u otro lenguaje de *scripting*. Para ver a qué nos referimos, consideremos la página http://mlb.mlb.com/milb/stats/stats.jsp?t=l_bat&y=2021&sid=l132&lid=132, que tiene los resultados de la Liga de Béisbol Mexicana.

Descargando la página con `requests.get` y convirtiéndola a un objeto de `bs4`:

In [163]:
page = requests.get('http://mlb.mlb.com/milb/stats/stats.jsp?t=l_bat&y=2021&sid=l132&lid=132')
soup = BeautifulSoup(page.content)

Sabemos que hay una tabla, así que la buscamos con `find`:

In [164]:
soup.find('table')

Sin embargo, esto no nos genera ningún resultado, lo cual significa que la tabla no está en el HTML que descargamos, y por lo tanto se genera de manera dinámica.

Para solucionar esto, podemos utilizar la librería `Selenium`. Esta esencialmente le da el control a Python de nuestro navegador web, con el cual puede accesar a páginas y lidiar con el contenido dinámico.

Para empezar, el objeto de driver:

In [165]:
from selenium import webdriver

Para generar el objeto navegador, necesitamos descargar un `chromedriver` de https://chromedriver.chromium.org/downloads, y ponerlo en algún lugar de nuestra computadora. Esto es lo que le permitirá a Selenium controlar nuestro navegador (de Chrome). Es importante notar que la versión del `chromedriver` que descarguemos debe de ser la misma que la de nuestro navegador.

Una vez hecho esto, simplemente lo inicializamos:

In [166]:
driver = webdriver.Chrome('/home/bondrewd/chromedriver')

Una ventana de Chrome debería de abrirse. Luego, accesamos la página:

In [167]:
driver.get('http://mlb.mlb.com/milb/stats/stats.jsp?t=l_bat&y=2021&sid=l132&lid=132')

Si bien el driver tiene funciones similares a las de `bs4` para hacer el scraping, es más fácil descargar la página completa, cerrar el navegador, y procesarla con `bs4`:

In [168]:
soup = BeautifulSoup(driver.page_source)
driver.close()

Si ahora intentamos buscar la tabla:

In [169]:
table = soup.find('table')
table

<table cellspacing="0" id="_628051636653301869"><col class="dg-name_display_first_last"/><col class="dg-player_id" style="display: none;"/><col class="dg-team_abbrev"/><col class="dg-pos"/><col class="dg-g"/><col class="dg-ab"/><col class="dg-r"/><col class="dg-h"/><col class="dg-d"/><col class="dg-t"/><col class="dg-hr"/><col class="dg-rbi"/><col class="dg-tb"/><col class="dg-bb"/><col class="dg-so"/><col class="dg-sb"/><col class="dg-cs"/><col class="dg-obp"/><col class="dg-slg"/><col class="dg-avg"/><col class="dg-ops"/><thead><tr><th class="dg-name_display_first_last" index="0" tabindex="0">NAME</th><th class="dg-player_id sortable" index="1" style="display: none;" tabindex="0"><span class="sortIcons">▲</span></th><th class="dg-team_abbrev" index="2" tabindex="0">TEAM</th><th class="dg-pos" index="3" tabindex="0"><abbr title="Position">POS</abbr></th><th class="dg-g sortable" index="4" tabindex="0"><abbr title="Games">G</abbr><span class="sortIcons">▼</span></th><th class="dg-ab so

Así, podemos procesarla de la misma manera que en el caso anterior:

In [170]:
rows = table.find_all('tr')

header = rows[0].find_all('th')
header = [cell.text.strip() for cell in header]

out = []
for row in rows[1:]:
    cells = row.find_all('td')
    cells_text = [cell.text.strip() for cell in cells]
    out.append(cells_text)

Convirtiéndola a una DataFrame:

In [171]:
df = pd.DataFrame(out, columns=header)
df

,NAME,▲,TEAM,POS,G▼,AB▼,R▼,H▼,2B▼,3B▼,...,RBI▼,TB▼,BB▼,SO▼,SB▼,CS▼,OBP▼,SLG▼,AVG▼,OPS▼
0,Tirso Ornelas,672359,NAV,LF,30,116,21,45,8,2,...,18,60,7,12,0,2,.423,.517,.388,.940
1,Victor Mendoza,628013,OBR,1B,28,106,11,37,5,0,...,20,54,7,14,2,1,.397,.509,.349,.906
2,Yadir Drake,660436,GSV,OF,29,94,14,32,4,0,...,18,42,20,14,2,0,.457,.447,.340,.904
3,Miguel Guzman,634739,GSV,3B,28,106,15,36,3,0,...,9,39,4,12,1,3,.372,.368,.340,.740
4,Sebastian Elizalde,590271,CUL,OF,27,99,15,33,4,1,...,20,42,17,8,11,5,.417,.424,.333,.841
5,Nick Torres,657051,HER,OF,27,106,16,35,5,1,...,21,60,11,37,2,2,.390,.566,.330,.956
6,Anthony Giansanti,572868,MTY,OF,28,97,19,32,5,0,...,13,49,18,20,2,1,.437,.505,.330,.942
7,Samar Leyva,653001,NAV,SS,28,97,11,32,8,4,...,9,57,2,10,2,2,.359,.588,.330,.947
8,Norberto Obeso,650995,HER,OF,25,97,14,32,4,0,...,9,36,17,7,3,2,.430,.371,.330,.801
9,Japhet Amador,515189,JAL,1B,30,122,14,40,4,0,...,20,59,7,16,0,0,.362,.484,.328,.845


También podemos descargar páginas sin necesidad de mostrar el navegador; a esto se le conoce como modo *headless*:

In [ ]:
from selenium.webdriver.chrome.options import Options

In [ ]:
chrome_options = Options()  
chrome_options.add_argument("--headless")

Es importante cerrar la sesión cuando acabemos de utilizarla, ya que de otra manera estará consumiendo recursos en el fondo. Para cerrarla automáticamente cuando acabe la ejecución, podemos utilizar el contexto `with`:

In [ ]:
with webdriver.Chrome('/home/bondrewd/chromedriver', options=chrome_options) as driver:
    driver.get('http://mlb.mlb.com/milb/stats/stats.jsp?t=l_bat&y=2021&sid=l132&lid=132')
    soup = BeautifulSoup(driver.page_source)

Los resultados son los mismos:

In [ ]:
table = soup.find('table')

rows = table.find_all('tr')

header = rows[0].find_all('th')
header = [cell.text.strip() for cell in header]

out = []
for row in rows[1:]:
    cells = row.find_all('td')
    cells_text = [cell.text.strip() for cell in cells]
    out.append(cells_text)
    
df = pd.DataFrame(out, columns=header)
df